# Praca domowa 


Karolina Stolarczyk 277053 


## Wprowadzenie


Celem poniższej pracy domowej było pobranie danych dotyczących podróży, nastepnie przygotowanie na podstawie tych danych 5 zapytań *sql*. Mieliśmy to wykonać za pomoca funkcji *read_sql_query* oraz za pomocą *zwykłych* funkcji dostepnych w bibliotece *pandas*. 


## Wykonanie


Do wykonania tego zadanie potrzebne były poniższe biblioteki:

In [529]:
import os, os.path
import sqlite3
import tempfile
import pandas as pd
import numpy as np
import timeit
import random

Następnym krokiem było wczytanie pobranych danych oraz połączenie z bazą.

In [388]:
Tags = pd.read_csv("D:/Studia/Python/Tags.csv.gz",compression = 'gzip')
Comments = pd.read_csv("D:/Studia/Python/Comments.csv.gz",compression = 'gzip')
PostLinks = pd.read_csv("D:/Studia/Python/PostLinks.csv.gz",compression = 'gzip')
Posts = pd.read_csv("D:/Studia/Python/Posts.csv.gz",compression = 'gzip')
Users = pd.read_csv("D:/Studia/Python/Users.csv.gz",compression = 'gzip')
Votes = pd.read_csv("D:/Studia/Python/Votes.csv.gz",compression = 'gzip')
Badges = pd.read_csv("D:/Studia/Python/Badges.csv.gz",compression = 'gzip')

In [19]:
# sciezka dostepu do bazy danych:
baza = os.path.join(tempfile.mkdtemp(), 'pd_2.db')
if os.path.isfile(baza): # jesli baza juz istneje...
    os.remove(baza) # ...usuniemy ja
conn = sqlite3.connect(baza) # połaczenie do bazy danych

Badges.to_sql("Badges", conn) # importujemy ramke danych do bazy danych
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

### Zadanie 1


Pierwsze polecenie *sql* wybiera tytuły z liczbą wpisów na ich temat oraz porządkuje je malejąco w zależności od liczby wpisów. Dane do tego zapytania pochodzą z *PostLinks* i *Posts*.

Wczytanie zapytania za pomoca funkcji *pd.read_sql_query* .

In [538]:
starttime = timeit.default_timer()

df1 = pd.read_sql_query("""
    SELECT Posts.Title, RelatedTab.NumLinks
    FROM
    (SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
        FROM PostLinks
        GROUP BY RelatedPostId) AS RelatedTab
    JOIN Posts ON RelatedTab.PostId=Posts.Id
    WHERE Posts.PostTypeId=1
    ORDER BY NumLinks DESC
""", conn)

czas_df1 = timeit.default_timer() - starttime

Wykonanie zadanie za pomocą funkcji.

In [563]:
starttime = timeit.default_timer()

#Wyznaczenie tabeli RelatedTab
RelatedTab = (PostLinks.groupby(by=['RelatedPostId'])[['RelatedPostId']].
              aggregate(['count']).reset_index(level='RelatedPostId'))

#Zmiana nazw kolumn
RelatedTab.columns = ['PostId', 'NumLinks']

#Zjoinowanie tabeli RelatedTab z tabelą Posts
Merged = pd.merge(RelatedTab, Posts[Posts.PostTypeId == 1], how='inner', left_on='PostId', right_on='Id')

#Wybranie odpowiednich kolumn i posortowanie wierszy, otrzymujemy Result - tabele końcową
Result = Merged[['Title','NumLinks']].sort_values('NumLinks', ascending= False).reset_index(drop=True)

czas_Result = timeit.default_timer() - starttime

Porównanie zawartości otrzymanych wyników. Nie chcemy, żeby permutacje wierszy miały wpływ podczas porównania, dlatego zresetujemy indeksy i posortujemy je od nowa, według takiego samego porządku. 

In [540]:
df1_sorted = df1.sort_values(['NumLinks','Title'], ascending= False).reset_index(drop=True)

Result_sorted = Result.sort_values(['NumLinks','Title'], ascending= False).reset_index(drop=True)

df1_sorted.equals(Result_sorted)

True

Otrzymaliśmy $True$, zatem otrzymane wyniki są równoważne. 

Teraz porównajmy czas  trwania tych zapytań. 

Otrzymane czasy mamy poniżej. Widzimy, że czas wykonania za pomoca funkcji * pd.read_sql_query* jest około 4 razy dłuższy, niż drugim sposobem.

In [541]:
[czas_df1, czas_Result]

[0.2039264000050025, 0.059033800003817305]

## Zadanie 2



Drugie polecenie *sql* wybiera informacje o dziesięciu użytkowanikach, którzy polubili najwiekszą liczbę postów.  Wybrane informacje o tych użytkownikach to nazwa, wiek, lokalizacja, liczba polubień, najbardziej lubiany post i liczba jego polubień. Brane pod uwage były tylko posty będace pytaniami. Dane do tego zapytania pochodzą z *Posts*.

Wczytanie zapytania za pomoca funkcji *pd.read_sql_query* .

In [542]:
starttime = timeit.default_timer()

df2 = pd.read_sql_query("""
    SELECT
        Users.DisplayName,
        Users.Age,
        Users.Location,
        SUM(Posts.FavoriteCount) AS FavoriteTotal,
        Posts.Title AS MostFavoriteQuestion,
        MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
    FROM Posts
    JOIN Users ON Users.Id=Posts.OwnerUserId
    WHERE Posts.PostTypeId=1
    GROUP BY OwnerUserId
    ORDER BY FavoriteTotal DESC
    LIMIT 10
""", conn)

czas_df2 = timeit.default_timer() - starttime

Wykonanie zadanie za pomocą funkcji.

In [566]:
starttime = timeit.default_timer()

#Utworzenie tabeli Posts2 zawierającej okreslone wiersze
Posts2 = Posts[Posts.PostTypeId ==1 & Posts.FavoriteCount.notnull()]

#Wyznaczenie tabelki z wartosciami sum i max FavoriteCount
SumMax = (Posts2.groupby(by=['OwnerUserId'],as_index=True)[['FavoriteCount']].
          aggregate(['sum','max']).reset_index(level='OwnerUserId'))
          
#Zamiana nazw kolumn         
SumMax.columns = ['OwnerUserId','FavoriteTotal', 'MostFavoriteQuestionLikes']
          
#Wyznaczenie tabelki powstałej z połączenia dwóch powyzszych
Posts3 = (pd.merge( SumMax, Posts2, how='left', left_on=['OwnerUserId','MostFavoriteQuestionLikes'], 
        right_on=['OwnerUserId','FavoriteCount'])[['OwnerUserId','FavoriteTotal','MostFavoriteQuestionLikes','Title']])

#Zamiana nazw kolumn
Posts3.columns = ['OwnerUserId','FavoriteTotal', 'MostFavoriteQuestionLikes','MostFavoriteQuestion']

.x
Merged = pd.merge(Posts3, Users[["DisplayName","Age","Location","Id"]], how='left', 
                  left_on='OwnerUserId', right_on='Id')

#Wybranie kolumn i uporzadkowanie ich
Result = (Merged[["DisplayName","Age", "Location", "FavoriteTotal", "MostFavoriteQuestion", 
                  "MostFavoriteQuestionLikes"]].sort_values('FavoriteTotal', ascending= False)[0:10])
          
czas_Result = timeit.default_timer() - starttime

Porównanie zawartości otrzymanych wyników. Nie chcemy, żeby permutacje wierszy miały wpływ podczas porównania, dlatego zresetujemy indeksy i posortujemy je od nowa, według takiego samego porządku. 

In [544]:
df2_sorted = df2.sort_values([
    "DisplayName","Age", "Location", "FavoriteTotal", "MostFavoriteQuestion", "MostFavoriteQuestionLikes"], 
    ascending= False).reset_index(drop=True)

Result_sorted = Result.sort_values([
    "DisplayName","Age", "Location", "FavoriteTotal", "MostFavoriteQuestion", "MostFavoriteQuestionLikes"], 
    ascending= False).reset_index(drop=True)

df2_sorted.equals(Result_sorted)

True

Otrzymaliśmy $True$, zatem otrzymane wyniki są równoważne. 

### Teraz porównajmy czas  trwania tych zapytań. 

Otrzymane czasy mamy poniżej. Widzimy, że czas wykonania za pomoca funkcji * pd.read_sql_query* jest około 3 razy dłuższy, niż drugim sposobem.

In [545]:
[czas_df2, czas_Result]

[0.3418999999994412, 0.09591380000347272]

### Zadanie3



Trzecie polecenie *sql* wybiera informacje o dziesięciu najbardziej komentowanych postach oraz liczbę tych komantarzy.  Brane pod uwage były tylko posty będace pytaniami. 

Wczytanie zapytania za pomoca funkcji *pd.read_sql_query* .

In [546]:
starttime = timeit.default_timer()

df3 = pd.read_sql_query("""
    SELECT
        Posts.Title,
        CmtTotScr.CommentsTotalScore
        FROM 
        (
        SELECT
            PostID,
            UserID,
            SUM(Score) AS CommentsTotalScore
        FROM Comments
        GROUP BY PostID, UserID
        ) AS CmtTotScr
    JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
    WHERE Posts.PostTypeId=1
    ORDER BY CmtTotScr.CommentsTotalScore DESC
    LIMIT 10
""", conn)

czas_df3 = timeit.default_timer() - starttime

Wykonanie zadanie za pomocą funkcji.

In [547]:
starttime = timeit.default_timer()

#Stworzenie tabelki CmtTotScr
CmtTotScr = (Comments.groupby(by=['PostId', 'UserId'],as_index=True)[['Score']].
             aggregate(['sum']).reset_index(level=['PostId', 'UserId']))

#Zamiana kolumn
CmtTotScr.columns = ['PostId', 'UserId','CommentsTotalScore']

#Utworzenie pomocniczej tabelki Posts2
Posts2 = Posts[Posts.PostTypeId == 1]

#Połączenie dwóch powyższych tabelek
Posts3 = pd.merge( CmtTotScr, Posts2, how='inner', left_on=['PostId','UserId'], right_on=['Id','OwnerUserId'])

#Uporzadkowanie i wybranie odpowiednich kolumn
Result = (Posts3[['Title', 'CommentsTotalScore']].
          sort_values('CommentsTotalScore', ascending= False)[0:10].reset_index(drop=True))

czas_Result = timeit.default_timer() - starttime

Porównanie zawartości otrzymanych wyników. Nie chcemy, żeby permutacje wierszy miały wpływ podczas porównania, dlatego zresetujemy indeksy i posortujemy je od nowa, według takiego samego porządku. 

In [548]:
df3_sorted = df3.sort_values(['CommentsTotalScore','Title'], ascending= False).reset_index(drop=True)

Result_sorted = Result.sort_values(['CommentsTotalScore','Title'], ascending= False).reset_index(drop=True)

df3_sorted.equals(Result_sorted)

True

Otrzymaliśmy $True$, zatem otrzymane wyniki są równoważne. 

Teraz porównajmy czas  trwania tych zapytań. 

Otrzymane czasy mamy poniżej. Widzimy, że czas wykonania za pomoca funkcji * pd.read_sql_query* jest około 3 razy dłuższy, niż drugim sposobem.

In [549]:
[czas_df3, czas_Result]

[0.7614568999997573, 0.23944990000018151]

### Zadanie4




Czwarte zapytanie *sql* wybiera niezdublowane informacje na temat użytkowników. Jest to, numer id, nazwa, reputacja, wiek i lokalizacja, którzy otrzymali przyznaną $Klasę =  1$ od 2 do 10 razy.

Wczytanie zapytania za pomoca funkcji *pd.read_sql_query* .

In [550]:
starttime = timeit.default_timer()

df4 = pd.read_sql_query("""
    SELECT DISTINCT
        Users.Id,
        Users.DisplayName,
        Users.Reputation,
        Users.Age,
        Users.Location
    FROM (
        SELECT
            Name, UserID
        FROM Badges
        WHERE Name IN (
                        SELECT
                        Name
                        FROM Badges
                        WHERE Class=1
                        GROUP BY Name
                        HAVING COUNT(*) BETWEEN 2 AND 10
                        )
        AND Class=1
        ) AS ValuableBadges
    JOIN Users ON ValuableBadges.UserId=Users.Id
""", conn)

czas_df4 = timeit.default_timer() - starttime

Wykonanie zadanie za pomocą funkcji.

In [551]:
starttime = timeit.default_timer()
#Stworzenie tabelki BadgesName
BadgesName = Badges[Badges.Class==1 ].groupby(by='Name')[['Name']].aggregate(['count']).reset_index(level='Name')

#Zamiana nazw kolumn
BadgesName.columns = ['Name', 'Count']

#Wybranie wierszy zgodnie z założeniami
BadgesName = BadgesName[(BadgesName.Count<10) & (BadgesName.Count>1)]

#Stworzenie tabelki ValuableBadges
ValuableBadges = pd.merge( Badges[Badges.Class==1], BadgesName, how='inner', left_on=['Name'], right_on=['Name'])

#Utworzenie tabelki wynikowej, wybranie kolumn i uporzadkowanie wierszy
Result = pd.merge( ValuableBadges, Users, how='inner', left_on=['UserId'], right_on=['Id'])[[
            'Id_y','DisplayName','Reputation','Age','Location']].drop_duplicates().reset_index(drop=True)

#Poprawienie nazwy kolumny
Result = Result.rename(columns={'Id_y':'Id'})

czas_Result = timeit.default_timer() - starttime

Porównanie zawartości otrzymanych wyników. Nie chcemy, żeby permutacje wierszy miały wpływ podczas porównania, dlatego zresetujemy indeksy i posortujemy je od nowa, według takiego samego porządku. 

In [552]:
df4_sorted = (df4.sort_values(['Id','DisplayName','Reputation','Age','Location'], ascending= False).
              reset_index(drop=True))

Result_sorted = (Result.sort_values(['Id','DisplayName','Reputation','Age','Location'], ascending= False).
                 reset_index(drop=True))

df4_sorted.equals(Result_sorted)

True

Otrzymaliśmy $True$, zatem otrzymane wyniki są równoważne. 

Teraz porównajmy czas  trwania tych zapytań. 

Otrzymane czasy mamy poniżej. Widzimy, że czas wykonania za pomoca funkcji * pd.read_sql_query* jest około 8 razy dłuższy, niż drugim sposobem.

In [553]:
[czas_df4, czas_Result]

[0.32294589999946766, 0.045879500001319684]

### Zadanie5



Piąte zapytanie *sql* wybiera tytuły postów wraz z indeksami oraz liczbę punktów za najlepszą odpowiedź, odpowiedź do zaakceptowania, a także różnicę w punktach dotyczącą tych odpowiedzi. Wyniki są uszeregowane malejąco. 

Wczytanie zapytania za pomoca funkcji *pd.read_sql_query* .

In [554]:
starttime = timeit.default_timer()

df5 = pd.read_sql_query("""
    SELECT
        Questions.Id,
        Questions.Title,
        BestAnswers.MaxScore,
        Posts.Score AS AcceptedScore,
        BestAnswers.MaxScore-Posts.Score AS Difference
    FROM (
        SELECT Id, ParentId, MAX(Score) AS MaxScore
        FROM Posts
        WHERE PostTypeId==2
        GROUP BY ParentId
        ) AS BestAnswers
    JOIN (
        SELECT * FROM Posts
        WHERE PostTypeId==1
        ) AS Questions
    ON Questions.Id=BestAnswers.ParentId
    JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
    WHERE Difference>50
    ORDER BY Difference DESC
""", conn)

czas_df5 = timeit.default_timer() - starttime

Wykonanie zadanie za pomocą funkcji.

In [555]:
starttime = timeit.default_timer()

# Tworzymy tabelkę pomocniczą z wybranymi kolumnami i o PostTypeId==2
PostsAns = Posts[Posts.PostTypeId==2][['ParentId','Id','Score']]

#Grupujemy ją by otrzymac tabelkę  BestAnswers
BestAnswers = (PostsAns.groupby(by=['ParentId']).aggregate({'Id':['first'],'Score':['max']}).
               reset_index(level='ParentId'))

#Poprawiamy nazwy kolumn
BestAnswers.columns = ['ParentId','Id','MaxScore']

#Tworzymy tabelkę po zjoinowaniu BestAnswers i Questions
BestQuestions = (pd.merge( BestAnswers, Posts[Posts.PostTypeId==1][['Id','Title','AcceptedAnswerId']], 
                          how='inner', left_on=['ParentId'], right_on=['Id']))

#Poprawiamy nazwę kolumny
BestQuestions = BestQuestions.rename(columns={'Id_x':'Id'})

#Do poprzedniej tabelki dojoinowujemy Posts
BestQuestionsPosts = (pd.merge(BestQuestions, Posts[['Id','Score']], how='inner', 
                               left_on=['AcceptedAnswerId'], right_on=['Id']))

#Dodajemy kolumne Difference
BestQuestionsPosts['Difference'] = BestQuestionsPosts['MaxScore'] - BestQuestionsPosts['Score']

#Znowu poprawiamy nazwy kolumn
BestQuestionsPosts = BestQuestionsPosts.rename(columns={'Id_x':'Id','Score':'AcceptedScore'})

#wybieramy te z Difference> 50, desc i zmieniamy indeksowanie
Result = (BestQuestionsPosts[BestQuestionsPosts.Difference >50].s
          ort_values('Difference', ascending= False).reset_index(drop=True))

#Wybieramy ostateczne kolumny i gotowe
Result = Result[['Id','Title','MaxScore','AcceptedScore','Difference']]

czas_Result = timeit.default_timer() - starttime

Porównanie zawartości otrzymanych wyników. Nie chcemy, żeby permutacje wierszy miały wpływ podczas porównania, dlatego zresetujemy indeksy i posortujemy je od nowa, według takiego samego porządku. 

In [556]:
df5_sorted = df5[['Title','MaxScore','AcceptedScore','Difference']]

Result_sorted = Result[['Title','MaxScore','AcceptedScore','Difference']]

df5_sorted.equals(Result_sorted)

True

Otrzymaliśmy $True$, zatem otrzymane wyniki są równoważne. 

Teraz porównajmy czas  trwania tych zapytań. 

Otrzymane czasy mamy poniżej. Widzimy, że czas wykonania za pomoca funkcji * pd.read_sql_query* jest około 2 razy dłuższy, niż drugim sposobem.

In [557]:
[czas_df5, czas_Result]

[0.37124640000547515, 0.17893570000160253]

Zamknięcie bazy.

In [558]:
conn.close()

## Podsumowanie

Wszystkie 5 zapytań *sql* udało nam się wykonać za pomocą funkcji z biblioteki *Pandas*. W każdym przypadku otrzymane wyniki były równoważne. Porównując czasy działania tych poleceń dużo szybciej zadziałały *zwykłe* funkcjie niż funkcja *pd.read_sql_query*.